Д# PABBO Optimization - Простой эксперимент

Этот ноутбук демонстрирует базовый процесс работы с PABBO:
1. Предобучение модели на синтетических 1D функциях (GP1D)
2. Оценка модели на тестовых функциях

## Шаг 1: Настройка окружения

In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt

# Добавляем путь к модулю pabbo_method
sys.path.append('../pabbo_method')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Current working directory: {os.getcwd()}")

## Шаг 2: Предобучение модели PABBO на GP1D

Согласно инструкции, сначала нужно предобучить модель PABBO на синтетических одномерных функциях (Gaussian Process 1D).

### Параметры предобучения:
- `data.name=GP1D` - использование синтетических GP данных
- `data.d_x=1` - одномерное пространство поиска
- `data.x_range="[[-1,1]]"` - диапазон значений x
- `data.min_num_ctx=1` - минимальное количество контекстных точек
- `data.max_num_ctx=50` - максимальное количество контекстных точек
- `train.num_query_points=100` - количество точек запроса
- `train.num_prediction_points=100` - количество точек предсказания
- `train.n_random_pairs=100` - количество случайных пар

In [ ]:
# Команда для запуска предобучения
# ВАЖНО: Эту команду нужно запускать из терминала, находясь в директории pabbo_method

train_command = """
cd ../pabbo_method && python train.py --config-name=train \
    experiment.expid=PABBO_GP1D \
    data.name=GP1D \
    data.d_x=1 \
    data.x_range="[[-1,1]]" \
    data.min_num_ctx=1 \
    data.max_num_ctx=50 \
    train.num_query_points=100 \
    train.num_prediction_points=100 \
    train.n_random_pairs=100 \
    experiment.device=cpu \
    experiment.wandb=false \
    train.n_steps=1000 \
    train.n_burnin=300
"""

print("Команда для запуска предобучения:")
print(train_command)
print("\n⚠️ ПРИМЕЧАНИЕ: Для быстрого теста мы используем:")
print("  - train.n_steps=1000 (вместо 8000 по умолчанию)")
print("  - train.n_burnin=300 (вместо 3000 по умолчанию)")
print("  - experiment.device=cpu (можно заменить на cuda если есть GPU)")
print("  - experiment.wandb=false (отключаем W&B для простоты)")
print("\nДля полноценного обучения увеличьте n_steps до 8000 и n_burnin до 3000")

### Запуск предобучения из ноутбука

Можно запустить прямо из ноутбука, но лучше использовать терминал для мониторинга процесса.

In [ ]:
# Раскомментируйте для запуска предобучения
# ВНИМАНИЕ: Это может занять время!

# import subprocess
# result = subprocess.run(
#     train_command,
#     shell=True,
#     capture_output=True,
#     text=True
# )
# print(result.stdout)
# if result.stderr:
#     print("STDERR:", result.stderr)

## Шаг 3: Проверка результатов предобучения

После завершения предобучения, чекпоинт модели должен сохраниться в:
`results/evaluation/PABBO/PABBO_GP1D/ckpt.tar`

In [ ]:
import os.path as osp

# Проверяем наличие чекпоинта
checkpoint_path = "../pabbo_method/results/evaluation/PABBO/PABBO_GP1D/ckpt.tar"

if osp.exists(checkpoint_path):
    print(f"✅ Чекпоинт найден: {checkpoint_path}")
    
    # Загружаем чекпоинт для проверки
    ckpt = torch.load(checkpoint_path, map_location='cpu')
    print(f"\nИнформация о чекпоинте:")
    print(f"  - Шаг обучения: {ckpt.get('step', 'N/A')}")
    print(f"  - Ключи: {list(ckpt.keys())}")
else:
    print(f"❌ Чекпоинт не найден: {checkpoint_path}")
    print("Необходимо сначала запустить предобучение (см. Шаг 2)")

## Шаг 4: Оценка модели на тестовой функции

После предобучения можно использовать модель для оптимизации различных функций.

### Пример: Оптимизация простой функции

Сначала попробуем на стандартной тестовой функции GP1D, затем перейдем к кастомным.

In [ ]:
# Команда для оценки на GP1D функции
eval_command = """
cd ../pabbo_method && python evaluate_continuous.py --config-name=evaluate \
    experiment.model=PABBO \
    experiment.expid=PABBO_GP1D \
    experiment.device=cpu \
    eval.eval_max_T=30 \
    eval.eval_num_query_points=256 \
    eval.num_parallel=1 \
    data.name=GP1D \
    data.d_x=1 \
    data.x_range="[[-1,1]]" \
    experiment.wandb=false \
    eval.num_seeds=5 \
    eval.num_datasets=5
"""

print("Команда для оценки модели:")
print(eval_command)
print("\n⚠️ Для быстрого теста используем:")
print("  - eval.num_seeds=5 (вместо 30)")
print("  - eval.num_datasets=5 (вместо 30)")

## Следующие шаги

После успешного предобучения и тестирования на GP1D, можно переходить к:

1. **Эксперимент 1**: Оптимизация функции $y(x) = \sin(x) + e^x$
   - Нужно добавить кастомную функцию в код
   - Запустить оценку на этой функции

2. **Эксперимент 2**: Оптимизация параметра T для LDA
   - Создать функцию, вычисляющую метрику LDA
   - Использовать `evaluate_discrete.py`

Смотрите следующие ноутбуки для детальных инструкций.

## Визуализация результатов

После запуска оценки, результаты сохраняются в директории results/

In [ ]:
# Функция для визуализации результатов
def plot_results(result_path):
    """
    Визуализация результатов оптимизации
    """
    if not osp.exists(result_path):
        print(f"Результаты не найдены: {result_path}")
        return
    
    # Загружаем метрики
    try:
        simple_regret = torch.load(f"{result_path}/SIMPLE_REGRET_S256_B1.pt")
        print(f"Simple regret shape: {simple_regret.shape}")
        
        # Построение графика
        plt.figure(figsize=(10, 6))
        mean_regret = simple_regret.mean(dim=(0, 1))
        std_regret = simple_regret.std(dim=(0, 1))
        
        steps = range(len(mean_regret))
        plt.plot(steps, mean_regret, label='Mean Simple Regret')
        plt.fill_between(steps, 
                         mean_regret - std_regret, 
                         mean_regret + std_regret, 
                         alpha=0.3)
        plt.xlabel('Optimization Step')
        plt.ylabel('Simple Regret')
        plt.title('PABBO Optimization Progress')
        plt.legend()
        plt.grid(True)
        plt.show()
        
    except Exception as e:
        print(f"Ошибка при загрузке результатов: {e}")

# Пример использования:
# plot_results("../pabbo_method/results/evaluation/GP1D/PABBO/PABBO_GP1D")